## Comp 6934 Winter 2025 Note 3

These notes are modified from notes by Dr. Terrance Tricco.

by E Brown.

In this notebook, we will introduce the pandas data library and focus on the following concepts:
- How to load data
- DataFrame and Series object types
- Explore data in pandas
- Access subsets of your data through filtering and slicing
- Sort your data
- Handle bad data (nulls, NaNs, etc)
- Using pandas in matplotlib
  
Data sets:

* <https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset>
    * You only need the CA file for this notebook examples
    * A reduced data set is provided with this notebook to help with loading and execution times.
* <https://www.kaggle.com/datasets/jash312/canada-daily-weather-data-from-2018-to-2023>
    

## Loading the pandas library

pandas is a data manipulation and analysis library. Typically pandas in imported under the symbol ``pd``. You should adopt this convention.

If you have installed the numpy package in your python environemnt, the installation should have included the pandas library.

In [ ]:
import pandas as pd

## Loading Data

Reading data is straightforward. pandas can read many data formats, such as .csv, .xlsx, and many more. The data format will be parsed and stores in a data structure know as a pandas **DataFrame**.

Note that this is quite a large data collection and could take a few moments to load.

In [ ]:
df = pd.read_csv('datasets/CA_videos.csv')
type(df) # what kind of object is it?

Before you start doing interesting things with your data, you should investigate your data set and make sure you understand its basics.

What is this data set that we've loaded? You can check the top rows of your data set using method ``head()`` and the bottom rows using ``tail()``.

In [ ]:
df.head()

In [ ]:
df.tail()

You can see that the DataFrame is effectively a two-dimensional table. There are rows of data, and each column has a title. The pandas DataFrame structure mimics similar data structures that have been found useful in other languages (R in particular)

The pandas dataframe has methods that are helpful for data analysis and exploration of the dataset, including methods to *nicely* display data in a text format.

This data set has information on YouTube videos. There are columns that have the title of the video, the video's channel, when it was first published, and when it became trending, along with other information that may or may not be useful or interesting.

## DataFrames and Series

You can ask for the column headers in the DataFrame:

In [ ]:
df.keys()

You can access the columns in your DataFrame by just indexing by its column name.

In [ ]:
df['title']

In [ ]:
s = df['title']

In [ ]:
type(s)

If DataFrames are like two-dimensional arrays, then **Series** are the pandas equivalent of one-dimensional arrays.

Series and DataFrames are closely related. It is helpful to think of your DataFrame as a bunch of Series glued together, with each column of your DataFrame being a Series.

## Data exploration

You have some data. Let's explore it a little bit to understand it better.

The best first steps are often using ``info()``, ``describe()``, ``value_counts()``, and ``unique()`` and ``nunique()``. Let's try these on our data set.

``info()`` is great for getting an overview on your data. 
- Are there missing values (nulls, NaNs)? 
- What data types are stored (int, float, bool, str, etc)?

In [ ]:
df.info()

``describe()`` will calculate statistics on the numeric data fields. It will return the average (mean), minimum and maximum values. 

It also returns the quartiles. These show the percentage of your data that is below each threshold. For example, the 25th percentile for views is approximately 419,000, meaning that 25% of the videos in the data set have fewer than this number of views.

In [ ]:
df.describe()

You can already see interesting stories start to emerge. 

Most videos have several hundred thousand to a few million views. The most viewed trending video has over 100 M! We see similar jumps in the order of magnitude of number of likes and comments for the top end videos.

These could also be an error in the data set, which occurs with alarming regularity, especially if you don't know how the data was collected.

``value_counts()`` is a very useful function. It will return the number of times each value has appeared in the data set.

In [ ]:
df['channel_title'].value_counts().head(20)

``nunique()`` returns the number of unique values present.

In [ ]:
df['channel_title'].nunique()

In [ ]:
df['video_id'].nunique()

For example, there are 3781 unique channels in the data set, out of a total of 13 194 (unique) trending videos. Therefore simple math says that the channels in this data have about 3.5 trending videos on average.

``unique()`` returns a list of all the unique values in the data. For example, calling ``unique()`` on the channel titles will give a list of all the unique channel names. There will not be any duplicates in that list.

In [ ]:
s = df['channel_title'].unique()
type(s)

In this case, pandas has not returned a `Series`. Instead, it is a `numpy.ndarray`.

In [ ]:
s = df['channel_title'].unique()

We could convert this to a Series if we wanted (with a new index).

In [ ]:
r = pd.Series(s)
type(r)

In [ ]:
r.head()

## Indexing - How to access your data

We have already used one way to access data in the DataFrame. We can index using the column name with subscript indexing syntax:

In [ ]:
df['views']

Unlike python dictionaries, you can also access DataFrame columns using the dot attribute selector operator:

In [ ]:
df.views

These two accessors produce the same value, but you might have to use the subscript syntax if your column name includes spaces, periods or other special characters, or if your column name is stored in a string variable or is a calculated expression.

You can also use a list of column names, instead of a single name.

In [ ]:
columns = ['title', 'channel_title', 'views']

df[columns]

In [ ]:
columns = ['views', 'title']

df[columns]

**The above methods are the most common ways to access your data.**

## Filtering and Slicing - Conditional Access

Filtering and slicing your data is easier in pandas than with standard python structures. You can easily use conditionals to filter down to specific subsets of your data.

What if we want only the videos that have more than 80M views?

In [ ]:
df[df['views'] > 80000000]

We can look at one particular video. And learn that this video was trending for 7 days in total.

In [ ]:
df[df['video_id'] == 'CocEMWdc7Ck']

We can also do more general string comparisons. Using ``.str``, we can access a number of string functions, such as ``.str.contains()``.

There are a large number of other string operations, like ``.str.endswith()``, ``.str.lower()``, ``.str.split()``, etc. These can be found in the API documentation on Series: https://pandas.pydata.org/pandas-docs/stable/reference/series.html

In [ ]:
df[df['title'].str.contains('Barbie')]

As well as filtering your data set by a list of values.

In [ ]:
df[df.channel_title.str.contains('Kurzgesagt')]

In [ ]:
science_channels = ['Kurzgesagt – In a Nutshell', 'Veritasium', 'SmarterEveryDay']

df[df['channel_title'].isin(science_channels)]

And you can combine filters.

Let's find trending videos that have both 'Avengers' in their title *and* have more than 50M views.

In [ ]:
df[(df['title'].str.contains('Barbie')) & (df['views'] > 10000000)]

Let's find trending videos that have more than 20M likes *or* more than 750k comments.

In [ ]:
df[(df['likes'] > 20000000) | (df['comment_count'] > 750000)]

In [ ]:
len(df)

**Important:** The above slicing and filtering does not modify the original DataFrame! They return new DataFrame objects. You will need to assign them if you want to keep the result.

In [ ]:
df[df.title.str.contains('Barbie')]

In [ ]:
df

In [ ]:
df_barbie = df[(df['title'].str.contains('Barbie')) & (df['views'] > 10000000)]

In [ ]:
df_barbie

## Integer and Index based accessors

pandas does include other ways to index into your data. You may sometimes need to use them.

``.iloc[]`` accesses rows and columns by their integer position. ``.loc[]`` accesses rows and columns by the index value and column name.

In [ ]:
df.head()

In [ ]:
df.iloc[0]

In [ ]:
df.iloc[0, 0]

In [ ]:
df.iloc[4:7]

In [ ]:
df.iloc[1:3, 1:4]

``.loc[]`` works by specifying the labels of the index (the numbers on the far left) and the column names.

In [ ]:
df.loc[0]

In [ ]:
df.loc[0, 'video_id']

In [ ]:
df_tmp = df.loc[[1,2], ['video_id', 'title', 'channel_title']]

In [ ]:
df_tmp

In [ ]:
df_tmp.loc[1]

In [ ]:
df.head()

Keep in mind that ``.loc[]`` is *label-based* access. In this case, the index labels are also the row numbers (0, 1, 2, etc). That doesn't have to be the case. The index could take on any value. We will see examples of this later.

Index labels can also be obtained by using an appropriate filter.

In [ ]:
df.loc[df.channel_title == 'Warner Bros. Pictures', ['video_id', 'title', 'channel_title']]

Use loc (and iloc) to setting values on slices of the data.

In [ ]:
df.loc[df.channel_title == 'Warner Bros. Pictures', 'video_id'] = 5.0

Trying to set a value on a slice will throw a warning because the slicing operation returns a copy. Modifying the copy does not modify the original data. See the example below.

In [ ]:
df[df.channel_title == 'Warner Bros. Pictures']['video_id'] = 10.0

In [ ]:
df[df.channel_title == 'Warner Bros. Pictures']  # video_id not updated

Use loc to set values on slices of a DataFrame.

## Sorting



``sort_values()`` can re-order your DataFrame by whichever column you specify.

Import parameters are:
- ``by=`` which is the column name(s) you want to sort by.
- ``ascending=`` which takes a True/False for whether you want to sort in ascending or descending order.
- ``ignore_index=`` will reset the index.


In [ ]:
df.sort_values(by='views', ascending=False)  # index keeps original value

In [ ]:
df.sort_values(by='views', ascending=False, ignore_index=True)  # index is reset

## Assigning new data

We can create new columns in our data set quite straightforwardly. When you try to assign data to a column that doesn't exist, pandas will just create it for you.

In [ ]:
df.info()

In [ ]:
df['new_column'] = 5

In [ ]:
df.info()

In [ ]:
df[['new_column', 'views', 'likes']]

In the above case, we have created a new column in which every value is 5.

Let's create a column that is derived from the existing columns. In this case, we can calculate the ratio of likes to views.

In [ ]:
df['like_ratio'] = df['likes'] / df['views']

In [ ]:
df

# Nulls, NaNs and infs

Working with NaNs or infinite numbers can cause many problems. What can we do?

In [ ]:
df.info()

The description column has Null or NaN for 353 rows. The like_ratio column also has a number of infs caused by divide by 0 errors. 

In [ ]:
df['like_ratio'].max()

In [ ]:
df[df['like_ratio'] == df['like_ratio'].max()]

One option is to filter your data set to avoid bad data. 

pandas comes with ``isna()``, ``isnull()``, ``notna()``, ``notnull()``, ``dropna()`` and ``fillna()`` methods. We can use these to remove the bad data in the description column.

In [ ]:
df = df.dropna()

In [ ]:
df.info()

Calling ``dropna()`` has removed all rows where a NaN or Null is present.

**Important:** remember that filtering and slicing return new DataFrames. The original DataFrame is not modified. You will need to assign this DataFrame if you want to keep it. In this case, we will re-assign over our original DataFrame.

We should also remove the rows that have infinite values for the like_ratio.

In [ ]:
df = df[df['like_ratio'] != df['like_ratio'].max()]

In [ ]:
df.info()

In [ ]:
df['like_ratio'].max()

# Pandas makes matplotlib more convenient

The convenience of Pandas is that it provides methods and operations suited to record-oriented data analysis and data manipulation. Recent versions of matplotlib have included an option of selecting your data from Pandas columns/series instead having to extract them into python lists or other iterables.

We show an example of this using the data set from the in-class problem set 2.

In [ ]:
# read the csv data into a dataframe
df = pd.read_csv('datasets/jash312/NL_combined_weather_data.csv')
type(df) # what kind of object is it?

In [ ]:
df.info()


Note that pandas has conveted the integer and float values to the appropriate types in the DataFrame, which solves a problem we had with parsing this csv file.

Next, we do the data reduction using Pandas techniques that are not available in standard python data structures:

In [ ]:
j2020df = df[df['Station Name'].str.contains("ST. JOHN'S INTL") & (df['Year'] == 2020)
                    & (df['Month'] == 1)]
j2020df.info()

To use columns from a DataFrame, provide the column names instead of data points for x and y, and specifiy the DataFrame as a data source using the keyword parameter **data=** 

In [ ]:
import matplotlib.pyplot as plt

plt.plot('Day','Mean Temp (°C)', data=j2020df)
plt.show()

# Summary

Load data: ``df = pd.read_csv('filename.csv')``

Exploratory data analysis:
- ``df.info()``
- ``df.describe()``
- ``df['column'].value_counts()``
- ``df['column'].unique()``
- ``df['column'].nunique()``
   
Filtering and slicing:
- ``df[df['column'] > val]``
- ``df[df['column'].str.contains()]``
- ``df[(df['column'] > val) & (df['column2'] > val2)]``
- etc

Sorting data:
- ``df.sort_values(by='column_name', ascending=False/True)``

Bad data:
- ``df[df['column'].notna()]``
- and ``.isna()``, ``.notnull()``, ``.isnull()``
- ``df.dropna()``
- ``df.fillna()``

Matplotlib:
- ``plt.plot('xcolname', 'ycolname', data=DataFrame)``